In [3]:
from cryptography.fernet import Fernet
from PIL import Image
import io

def generate_key():
    return Fernet.generate_key()

def encrypt_image(key, input_file, output_image):
    cipher = Fernet(key)
    with open(input_file, 'rb') as file:
        data = file.read()
        encrypted_data = cipher.encrypt(data)

    # Determine the size of the image based on the length of encrypted data
    size = int(len(encrypted_data)**0.5) + 1

    # Pad the encrypted data with zeros to fill the square image
    padded_data = encrypted_data.ljust(size**2, b'\x00')

    # Create a 2D image from the padded data
    encrypted_image = Image.frombytes('L', (size, size), padded_data)

    # Save the image
    encrypted_image.save(output_image)

def decrypt_image(key, input_image, output_file):
    cipher = Fernet(key)

    # Open the image and get pixel data
    encrypted_image = Image.open(input_image)
    encrypted_data = encrypted_image.tobytes()

    # Decrypt the data
    decrypted_data = cipher.decrypt(encrypted_data)

    # Save the decrypted data to a file
    with open(output_file, 'wb') as file:
        file.write(decrypted_data)

# Example usage
key = generate_key()
encrypt_image(key, './combined_images/positive_images/AESDD__joy__positive__1_mel_spectrogram.png', 'encrypted_image.png')
decrypt_image(key, 'encrypted_image.png', 'decrypted_image.png')

In [13]:
from cryptography.fernet import Fernet
from PIL import Image
import io
import glob
import os 
from tqdm import tqdm

key = generate_key()

positive_images = [image.replace(os.sep, '/') for image in glob.glob('./combined_images/positive_images/*')]
negative_images = [image.replace(os.sep, '/') for image in glob.glob('./combined_images/negative_images/*')]

for positive_image in tqdm(positive_images):
    encrypted_image = positive_image.replace('combined_images', 'encrypted_images')
    encrypted_image = '.'.join(encrypted_image.split('.')[:-1]) + '__encrypted.' + encrypted_image.split('.')[-1]
    encrypt_image(key, positive_image, encrypted_image)
    
for negative_image in tqdm(negative_images):
    encrypted_image = negative_image.replace('combined_images', 'encrypted_images')
    encrypted_image = '.'.join(encrypted_image.split('.')[:-1]) + '__encrypted.' + encrypted_image.split('.')[-1]
    encrypt_image(key, negative_image, encrypted_image)

100%|████████████████████████████████████████████████████████████████████████████| 21915/21915 [07:56<00:00, 46.02it/s]


In [1]:
from PIL import Image
import numpy as np
import random

def encrypt_image(original_image_path, encrypted_image_path, key):
    # Open the image
    img = Image.open(original_image_path)
    
    # Convert the image to a NumPy array
    img_array = np.array(img)

    # Get the shape of the array
    height, width, channels = img_array.shape

    # Flatten the array
    flat_array = img_array.flatten()

    # Use a secure random key
    random.seed(key)

    # Shuffle the pixel indices
    indices = list(range(len(flat_array)))
    random.shuffle(indices)

    # Permute the flattened array
    permuted_array = [flat_array[i] for i in indices]

    # Reshape the array back to the original image shape
    img_array = np.array(permuted_array).reshape((height, width, channels))

    # Create a new image from the modified array
    encrypted_img = Image.fromarray(img_array.astype('uint8'))

    # Save the encrypted image
    encrypted_img.save(encrypted_image_path)

# Example Usage
encrypt_image('./combined_images/positive_images/AESDD__joy__positive__1_mel_spectrogram.png', 'encrypted_image.png', key='secure_random_key')


In [34]:
import numpy as np
from PIL import Image
import random

def randomly_swap_columns(image_path, num_swaps=250):
    # Open the image using PIL
    img = Image.open(image_path)
    
    # Convert the image to a numpy array
    img_array = np.array(img)

    # Get the number of columns in the image
    num_columns = img_array.shape[1]

    for _ in range(num_swaps):
        # Generate two random column indices to swap
        column1, column2 = random.sample(range(num_columns), 2)

        # Swap the columns in the numpy array
        img_array[:, [column1, column2]] = img_array[:, [column2, column1]]

    # Create a new image from the modified array
    new_img = Image.fromarray(img_array)

    # Save or display the new image
    new_img.save("randomly_swapped_image.png")

# Example usage
image_path = './combined_images/positive_images/AESDD__joy__positive__1_mel_spectrogram.png'
randomly_swap_columns(image_path)


In [27]:
import librosa
import librosa.display
import numpy as np
from scipy.io.wavfile import write
import matplotlib.pyplot as plt
from PIL import Image

def mel_spectrogram_to_audio(mel_spectrogram_path, out_wav_path, sr=22050, n_fft=2048, hop_length=512, win_length=None):
    # Load the mel-spectrogram image
    mel_spectrogram_image = Image.open(mel_spectrogram_path)
    mel_spectrogram = np.array(mel_spectrogram_image)

    # Assume that the mel-spectrogram is in dB
    mel_spectrogram = librosa.db_to_power(mel_spectrogram)

    # Convert back to linear spectrogram
    spectrogram = librosa.feature.inverse.mel_to_stft(mel_spectrogram, sr=sr, n_fft=n_fft)

    # Invert the STFT
    y = librosa.griffinlim(spectrogram, hop_length=hop_length, win_length=win_length)
    
    y = y.astype(int)
    
    # # Save the result
    write(out_wav_path, sr, y)

# Example usage:
mel_spectrogram_path = './combined_images/positive_images/AESDD__joy__positive__1_mel_spectrogram.png'
out_wav_path = 'output_audio.wav'
mel_spectrogram_to_audio(mel_spectrogram_path, out_wav_path)

In [30]:
import numpy as np
from scipy.io.wavfile import write
import librosa

def mel_spectrogram_to_audio(mel_spectrogram_path, out_wav_path, sr=22050, n_fft=2048, hop_length=512, win_length=None):
    # Load the mel-spectrogram image
    # ... (rest of your loading code) ...
    
    # Load the mel-spectrogram image
    mel_spectrogram_image = Image.open(mel_spectrogram_path)
    mel_spectrogram = np.array(mel_spectrogram_image)

    # Assume that the mel-spectrogram is in dB
    mel_spectrogram = librosa.db_to_power(mel_spectrogram)

    # Convert back to linear spectrogram
    spectrogram = librosa.feature.inverse.mel_to_stft(mel_spectrogram, sr=sr, n_fft=n_fft)
    
    # Invert the STFT
    y = librosa.griffinlim(spectrogram, hop_length=hop_length, win_length=win_length)

    # Normalize the audio to the range [-1.0, 1.0]
    y = y / np.max(np.abs(y))

    # Convert to 16-bit PCM format
    y_16bit = np.int16(y * 32767)

    # Save the result
    write(out_wav_path, sr, y_16bit)

# Example usage
mel_spectrogram_path = './combined_images/positive_images/AESDD__joy__positive__1_mel_spectrogram.png'
out_wav_path = 'output_audio.wav'
mel_spectrogram_to_audio(mel_spectrogram_path, out_wav_path)

In [3]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
from pydub import AudioSegment
import os

AudioSegment.ffmpeg = "./ffmpeg/bin/ffmpeg.exe"
AudioSegment.ffprobe = "./ffmpeg/bin/ffprobe.exe"
AudioSegment.converter = "./ffmpeg/bin/ffmpeg.exe"

def encrypt_audio(input_audio_path, output_encrypted_audio_path, key):
    # Read the audio file
    audio = AudioSegment.from_file(input_audio_path)

    # Convert audio to binary data
    binary_data = audio.raw_data

    # Pad the binary data to be multiple of block size (AES block size is 128 bits)
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(binary_data) + padder.finalize()

    # Encrypt the binary data using AES
    cipher = Cipher(algorithms.AES(key), modes.CFB(os.urandom(16)), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    # Save the encrypted data as a new audio file
    encrypted_audio = AudioSegment(encrypted_data, frame_rate=audio.frame_rate, sample_width=audio.sample_width, channels=audio.channels)
    encrypted_audio.export(output_encrypted_audio_path, format="wav")

# Example Usage
key = os.urandom(32)  # 256-bit key for AES-256
encrypt_audio('./final_dataset/AESDD/positive/AESDD__joy__positive__1.wav', 'encrypted_audio.wav', key)

In [8]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding, hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import os
from pydub import AudioSegment

def encrypt_audio(input_audio_path, output_encrypted_audio_path, password):
    # Read the audio file using pydub
    audio = AudioSegment.from_file(input_audio_path)

    # Convert audio to binary data
    binary_data = audio.raw_data

    # Pad the binary data to be a multiple of the block size (AES block size is 128 bits)
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(binary_data) + padder.finalize()

    # Derive a key from the password
    salt = os.urandom(16)
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),  # Corrected attribute
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())

    # Encrypt the binary data using AES-GCM
    cipher = Cipher(algorithms.AES(key), modes.GCM(os.urandom(16)), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    # Save the encrypted data as a new audio file
    encrypted_audio = AudioSegment(ciphertext, frame_rate=audio.frame_rate, sample_width=audio.sample_width, channels=audio.channels)
    encrypted_audio.export(output_encrypted_audio_path, format="wav")

# Example Usage
key = "PASSWORD_TO_ENCRYPT" #os.urandom(32)  # 256-bit key for AES-256
encrypt_audio('./final_dataset/AESDD/negative/AESDD__anger__negative__1.wav', 'encrypted_audio.wav', key)